## Содержание
```
1 Прямой проход и обратное распространение
2 Разбиение набора данных
3 Нормализация данных
4 Валидация и переобучение
5 Схождение
6 Контрольные точки и ранняя остановка
7 Гиперпараметры
8 Инвариантность
9 Сырые наборы данных
10 Сохранение/Восстановление модели
```

## Прямой проход и обратное распространение

#### Прямой проход

<img src="images/forward_pass.png" alt="forward" height=60% width=60%>

```
    Данные подаются в нейронную сеть пакетами (batch-ами). Пакет может состоять из одного или нескольких выбранных наугад примеров из тренировочных данных.
    Тренировочные данные подаются в модель несколько раз. Всякий раз, когда мы подаем все тренировочные данные целиком, это
называется эпохой. Каждая эпоха представляет собой отличающуюся случайную перестановку примеров в пакетах – то есть не существует двух эпох с одинаковым порядком следования примеров.

```

#### Историческая справка
```
    В  ранних нейронных сетях, таких как персептрон и пр., исследователи экспериментировали со способами обновления весов, чтобы получать правильный ответ.
    Когда они работали всего с несколькими нейронами и простыми задачами, первым подходом было случайное обновление весов. Кто бы мог подумать, но случайная догадка работала. Однако, такой подход не масштабировался на большое количество нейронов (скажем, тысячи) и реально существующие приложения; на то, чтобы сделать правильную случайную догадку, могли уйти тысячи лет
    Следующим логическим шагом было попытаться сделать случайное значение пропорциональным удаленности от правильного значения. Другими словами, чем больше ошибка (удаленнее предсказание), тем больше диапазон случайных значений; и чем ближе, тем меньше диапазон. Это сокращает время поиска (весов нейронной сети) до сотен лет.
    Но такой подход не работал с многослойными нейронными сетями. Обнаружилось, что при наличии нескольких слоев эта методика приводит
к тому, что "левая рука" – один слой – отменяет работу "правой руки" – другого слоя.
```

#### Обратное распространение

<img src="images/back_prop.png" alt="backpropagation" height=60% width=60%>

```
    После того как каждый пакет тренировочных данных пропущен через модель в прямом направлении и вычислена потеря, потеря распространяется через модель в обратном направлении. Мы идем слой за слоем, обновляя параметры модели (веса и  смещения), начиная с  верхнего слоя (выхода) и  продвигаясь к  нижнему слою (входу).
    Общий метод обновления параметров основан на градиентном спуске. Оптимизатор – это имплементация градиентного спуска, задача которого состоит в обновлении параметров для минимизации потери (максимального приближения к  правильному ответу) в последующих пакетах.
```

## Разбиение набора данных

#### Тренировочный и тестовый наборы

<img src="images/train_test.png" alt="train_data" height=60% width=60%>

```
     По умолчанию будем рассматривать наборы данных, которые достаточно велики и разнообразны, чтобы представлять выборочное распределение (моделируемую популяцию), и очищены (не зашумлены).
```

```
    Имея в своем распоряжении набор данных, следующим шагом будет его разбиение на примеры, которые будут использоваться для тренировки, и те, которые будут использоваться для тестирования (т.н. отложенные). Мы тренируем модель той порцией набора данных, которые являются тренировочными. Если допустить, что тренировочные данные являются хорошим выборочным распределением, то точность на тренировочных данных должна отражать точность, получаемую при развертывании в реально существующей среде (на примерах, не встречавшихся моделью во время тренировки). Но как проверить истинность этого утверждения до того, как модель будет развернута? Отсюда и вытекает предназначение тестовых данных. Мы откладываем часть данных, с помощью которой будем тестировать модель, после того как она будет обучена, чтобы убедиться, что мы получили или не получили нужную точность.
    Сколько данных следует откладывать на тренировку и тестирование? Исторически сложилось эмпирическое соотношение 90/10: 90% для тренировки и 10% для тестирования.
```

#### Пример

```
    Для классифицирования изображений есть несколько хорошо известных, подготовленных наборов данных, таких как MNIST, CIFAR-10/100*, SVHN, Flowers, Cats vs. Dogs.
    Рассмотрим один из них

* аббр. от англ. Canadian Institute for Advanced Research – Канадский институт перспективных исследований

```

In [ ]:
import torch
import torchvision

# MNIST

# Downloading the MNIST dataset
train_dataset = torchvision.datasets.MNIST(
    root="./MNIST/train", train=True,
    transform=torchvision.transforms.ToTensor(),
    download=False)

test_dataset = torchvision.datasets.MNIST(
    root="./MNIST/test", train=False,
    transform=torchvision.transforms.ToTensor(),
    download=False)

```
train - 60000 imgs
test  - 10000 imgs
```

<img src="images/MNIST_data.png" alt="mnist_dataset" height=60% width=60%>

In [3]:
def encode_label(j):
    # 5 -> [[0], [0], [0], [0], [0], [1], [0], [0], [0], [0]]
    e = np.zeros((10,1))
    e[j] = 1.0
    return e

def reshape_data(data):
    features = [np.reshape(x[0][0].numpy(), (784,1)) for x in data]
    #print('features\n', len(features[0]))
    labels = [encode_label(y[1]) for y in data]
    #print('labels\n', len(labels[0]))
    return zip(features, labels)